# Introduction

This notebook accompanies my blog post on using prompt-based GPT for analyzing textual data. 

You can find the blog post here: <link>

In [2]:
NAME = 'prompt_based_gpt_example' 
PYTHON_VERSION = '3.10'

## Preamble

### Imports  

In [3]:
import re, math, time, sys, copy, random, json
from pathlib import Path
import pandas as pd
import numpy as np

### Settings

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

------
# Code
------

## Dataset

We will need some data to work with, so below are some fake employee reviews.  
Fun fact, most of these fake reviews are actually generated by GPT-3 through the OpenAI Playground. 

In [5]:
reviews_list = [
    {
        'review_id': 1, 
        'review_text': '''Plumbing Co is a great company to work for! The compensation is great and above the industry standard. The benefits are also very good. The company is very fair and treats its employees well. I would definitely recommend Plumbing Co to anyone looking for a great place to work.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'The compensation is great and above the industry standard.', 'sentiment': 'positive'},
            {'sentence': 'The benefits are also very good.', 'sentiment': 'positive'}
        ]
    },
    {
        'review_id': 2, 
        'review_text': '''I've been working at Plumbing Co for a few months now, and I've found it to be a pretty decent place to work. The salary is pretty average, but the coffee is really great. Overall, I think it's a pretty good company to work for. The hours are reasonable, and the work is fairly easy. I would recommend it to anyone looking for a decent job.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'The salary is pretty average, but the coffee is really great.', 'sentiment': 'neutral'},
        ]
    },
    {
        'review_id': 3, 
        'review_text': '''Plumbing Co is a great place to work for those who are interested in the field of plumbing. The company is always expanding and there is room for advancement. The pay is too low, however, and this is the only downside to working for Plumbing Co.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'The pay is too low, however, and this is the only downside to working for Plumbing Co.', 'sentiment': 'negative'},
        ]
    },
    {
        'review_id': 4, 
        'review_text': '''I had a great time working with Chairlift Brothers! They were very professional and compensated me well for my time. I would definitely recommend them to anyone looking for a great chairlift company.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'They were very professional and compensated me well for my time.', 'sentiment': 'positive'},
        ]
    },
    {
        'review_id': 5, 
        'review_text': '''The pay is good, and the benefits are great. I found the work at Plumbing Co to be very interesting and enjoyable. Their lunches could use improvement though, the salads are never fresh. I would definitely recommend this company to anyone looking for a great place to work.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'The pay is good, and the benefits are great.', 'sentiment': 'positive'},
        ]
    },
    {
        'review_id': 6, 
        'review_text': '''Stay away from this company!! Upper level management is horrible, very bad experience.''',
        'in_training' : True,
        'pay_sentences' : [
        ]
    },
    {
        'review_id': 7, 
        'review_text': '''The office buildings are quite nice. The cleaning lady on Thursday morning sometimes even brought me coffee, insane! I enjoyed the collegiality of the group and the short work hours. Salary was reasonable and the work hours were standard. Worth a look.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'Salary was reasonable and the work hours were standard.', 'sentiment': 'neutral'},
        ]
    },
    {
        'review_id': 8, 
        'review_text': '''I didn't enjoy my time working at Plumbing Co. The parking spaces are quite wide, which was great for my mini-van. The pay was too low, and the work was very boring. Joel from IT was great though, very helpful.''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'The pay was too low, and the work was very boring.', 'sentiment': 'negative'},
        ]
    },
    {
        'review_id': 9, 
        'review_text': '''The CEO, Terresa, was just a great human being, thanks for everything! The pay is great, and the benefits are great. The only reason I left was because I had the opportunity to become a YouTube celebrity. If you are looking to join a VR startup, this is the company! Did I mention the pay is great?''',
        'in_training' : True,
        'pay_sentences' : [
            {'sentence': 'The pay is great, and the benefits are great.', 'sentiment': 'positive'},
            {'sentence': 'Did I mention the pay is great?', 'sentiment': 'positive'},
        ]
    },
    {
        'review_id': 10, 
        'review_text': '''Nothing to mention. Standard company.''',
        'in_training' : True,
        'pay_sentences' : [
        ]
    },
    {
        'review_id': 11, 
        'review_text': '''I had a great time working with Chairlift Brothers! They were very professional and compensated me well for my time. I would definitely recommend them to anyone looking for a great chairlift company.''',
        'in_training' : False,
    },
    {
        'review_id': 12, 
        'review_text': '''I would not recommend working for Chairlift Brothers. They do not pay well and the hours are long. The work is also very physical and challenging, so it's not for everyone. There are better companies out there that will pay you more for your time and effort.''',
        'in_training' : False,
    },
    {
        'review_id': 13, 
        'review_text': '''The Chairlift Brothers company is terrible. They don't pay their employees well, and the work is extremely demanding. The company is also very disorganized, and it's hard to get anything done. Overall, I would not recommend working for this company.''',
        'in_training' : False,
    },
    {
        'review_id': 14, 
        'review_text': '''I don't love working at Company ABC, but the wage is decent and the benefits are decent. I would recommend this company to anyone looking for a decent place to work.''',
        'in_training' : False,
    }
]

## Create a function that can generate a prompt and completion

Designing a prompt and completion is important to (1) make sure the model solves your task and (2) to make the completions consistent so that we can parse them. You generally want to make your prompt+completion as short as possible as the speed and costs tend to scale linearly with the number of tokens. Shorter prompts means a higher throughput and lower cost.

We do have a few things that are required to include to make things behave consistently. You need to include:

1. A separator that indicates where your prompt ends and the completion begins. In the example below I use `\n####\n`. 
2. A seperator that indicates where the completion ends and the next prompt begins. In the example below I use `<|endoftext|>`. 

These seperators can be anything in most cases, however, you want to make sure that the seperator does not occur naturally in the text. 

You also want to make sure that the information in the completion is easy to parse. In the example below we accomplish this by forcing each sentence to be on a seperate line and we put the sentiment at the start of each sentence in arrow brackets. 

In [6]:
def generate_prompt_and_completion(review_data, prompt_end = "\n####\n", completion_end = "\n<|endoftext|>"):    
    ret_dict = {}
    ret_dict['prompt'] = review_data['review_text'] + prompt_end ## 

    if review_data['in_training']:
        completion_list = []
        for sentence in review_data['pay_sentences']:
            completion_list.append(f'''<{sentence['sentiment']}> {sentence['sentence']}''') 
        ret_dict['completion'] = '\n'.join(completion_list) + completion_end

    return ret_dict

#### Show an example

In [7]:
tmp = generate_prompt_and_completion(reviews_list[0])
print(tmp['prompt'] + tmp['completion'])

Plumbing Co is a great company to work for! The compensation is great and above the industry standard. The benefits are also very good. The company is very fair and treats its employees well. I would definitely recommend Plumbing Co to anyone looking for a great place to work.
####
<positive> The compensation is great and above the industry standard.
<positive> The benefits are also very good.
<|endoftext|>


#### Run every review through our generator

In [8]:
training_list = []
to_predict_list = []
for review in reviews_list:
    if review['in_training']:
        training_list.append(generate_prompt_and_completion(review))
    else:
        to_predict_list.append(generate_prompt_and_completion(review))

In [9]:
training_list[:4]

[{'prompt': 'Plumbing Co is a great company to work for! The compensation is great and above the industry standard. The benefits are also very good. The company is very fair and treats its employees well. I would definitely recommend Plumbing Co to anyone looking for a great place to work.\n####\n',
  'completion': '<positive> The compensation is great and above the industry standard.\n<positive> The benefits are also very good.\n<|endoftext|>'},
 {'prompt': "I've been working at Plumbing Co for a few months now, and I've found it to be a pretty decent place to work. The salary is pretty average, but the coffee is really great. Overall, I think it's a pretty good company to work for. The hours are reasonable, and the work is fairly easy. I would recommend it to anyone looking for a decent job.\n####\n",
  'completion': '<neutral> The salary is pretty average, but the coffee is really great.\n<|endoftext|>'},
 {'prompt': 'Plumbing Co is a great place to work for those who are interested

## Create a function to parse a completion string back into a Python object

The completion generated by our GPT model are in plain text, so we need to parse the text to extract the information we need back into a Python object. 

The easiest way to accomplish this is through one or multiple regular expressions.   
An important thing to realize is that you control what the completion will look like, so if it is difficult to parse you may want to change the way you design your completion in the previous step. 

In [1]:
def parse_completion(completion):
    compensation_sentences = re.findall(r'<(.*?)> (.*?)\n', completion) ## Our regular expression here is simple because we designed it conveniently!

    completion_dict = {
        'sentences' : compensation_sentences,
        'num_positive' : 0,
        'num_negative' : 0,
        'num_neutral' : 0,
        'num_sentences' : len(compensation_sentences)
    }

    for sen in compensation_sentences:
        if sen[0] == 'positive':
            completion_dict['num_positive'] += 1
        elif sen[0] == 'negative':
            completion_dict['num_negative'] += 1
        elif sen[0] == 'neutral':
            completion_dict['num_neutral']  += 1

    return completion_dict

In [10]:
completion = '<positive> The compensation is great and above the industry standard.\n<positive> The benefits are also very good.\n<|endoftext|>'
parse_completion(completion)

{'sentences': [('positive',
   'The compensation is great and above the industry standard.'),
  ('positive', 'The benefits are also very good.')],
 'num_positive': 2,
 'num_negative': 0,
 'num_neutral': 0,
 'num_sentences': 2}

-------
# Generating our predictions using OpenAI
-------

## A primer on using OpenAI

You can sign up for an OpenAI account here: https://beta.openai.com/overview   
They give you a small amount of free credit to use for your first experiments.

You can interact with OpenAI models in two ways:

1. You can use the Playground on their website and manually feed it prompts and generate completions.  
2. You can submit a prompt to their API from your code to programmatically generate completions. 

In the sections below I will demonstrate how to use the API.    
You can play around with the Playground here: https://beta.openai.com/playground

I will skip over a lot of details in the code below, so I strongly recommend also reading the OpenAI documentation:
https://beta.openai.com/docs

-------

#### Setting up your OpenAI API credentials

To interact with the API you need to follow the following steps:

1. Log in to the openAI website
2. Click on the "Personal" button in the top right corner and select "View API Keys"
3. Create a new secret key or copy an existing one
4. Install the OpenAI Python library by running `pip install openai`

You can now interact with the API using the OpenAI Python client and your secret key. 

**Two important warnings:**

**Warning 1:** Keep your secret key private! The secret key is like your password, if others get access to your secret key then can use OpenAI with your payment details. I recommend storing your secret key in an environment variable or loading it from a file, don't store it in your code if you can avoid it!

**Warning 2:** The OpenAI API is a paid service and every requests will use up your credits. Once you run out of free credits you will be charged for running predictions, these are generally very small amounts, but this can get out of hand if you don't pay attention, so code responsibly! You can set soft and hard limits in on the OpenAI website if you are worried about this.

In [12]:
import openai

In [13]:
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = input('Enter your OpenAI API key: ') ## This will set your API key in your environment variables if it is not already set.
openai.api_key = os.environ['OPENAI_API_KEY']

## Let's test the API and perform a zero shot prediction
-------------------

Something is considered a zero shot prediction if we don't give the general model any examples and just present it with our prompt.

In [14]:
zero_shot_prompt = training_list[0]['prompt']
print(zero_shot_prompt)

Plumbing Co is a great company to work for! The compensation is great and above the industry standard. The benefits are also very good. The company is very fair and treats its employees well. I would definitely recommend Plumbing Co to anyone looking for a great place to work.
####



#### API example

In [15]:
result = openai.Completion.create(
    model = 'davinci',           ## OpenAI provides multiple models "engines", see: https://beta.openai.com/docs/engines
    prompt = zero_shot_prompt,
    stop = ["<|endoftext|>"],    ## This tells OpenAI when to stop generating tokens.
    max_tokens =  500,           ## This is the maximum number of tokens to generate.
    temperature= 0.7             ## The temparature controls how "random" the generated tokens are, it roughly controls the creativity of the generations. 
)

In [16]:
print(result['choices'][0]['text'])

We are a small, but growing company in the Dallas/Ft Worth area looking for talented and driven individuals to join out team! If you are interested please contact our office at 972.847.8165 and ask for Rob or Eric.


As you can see, the above completion is nothing like the completions we are after.     
This is not surprising given that we did not show the model what type of completions we want it to generate, so it just made a guess.  

One thing we could try, which does sometimes work, is to add an explicit instruction to the model in addition to the prompt.    
The blog post includes an example of this and you would implement it by simply adding the instruction to the end of your prompt. 

# Let's give the model a few examples (i.e., a few shot prediction)
-------------------

By including a few example prompts + completions in our prompt we can better tell the model what we are expecting.

As you can see below, this implies simply adding a bunch of prompt+completions to the start of our prompt.   
This is simply to implement, however, our throughput speed and cost scale linearly with the number of tokens, and including the examples does introduce a lot of extra tokens!

In [19]:
few_shot_examples = ''
for training_item in training_list[:3]:
    few_shot_examples += training_item['prompt']
    few_shot_examples += training_item['completion'] + '\n'

In [20]:
few_shot_prompt = few_shot_examples + to_predict_list[0]['prompt']
print(few_shot_prompt)

Plumbing Co is a great company to work for! The compensation is great and above the industry standard. The benefits are also very good. The company is very fair and treats its employees well. I would definitely recommend Plumbing Co to anyone looking for a great place to work.
####
<positive> The compensation is great and above the industry standard.
<positive> The benefits are also very good.
<|endoftext|>
I've been working at Plumbing Co for a few months now, and I've found it to be a pretty decent place to work. The salary is pretty average, but the coffee is really great. Overall, I think it's a pretty good company to work for. The hours are reasonable, and the work is fairly easy. I would recommend it to anyone looking for a decent job.
####
<neutral> The salary is pretty average, but the coffee is really great.
<|endoftext|>
Plumbing Co is a great place to work for those who are interested in the field of plumbing. The company is always expanding and there is room for advancement

#### API example

In [21]:
result = openai.Completion.create(
    model = 'davinci', 
    prompt = few_shot_prompt,
    stop = ["<|endoftext|>"],
    max_tokens =  1000,      ## The number of tokens in the prompt also count towards the max_tokens, so we need to increase it a bit to accomodate the examples. 
    temperature= 0.7
)

In [22]:
completion = result['choices'][0]['text']
print(completion)

<positive> They were very professional and compensated me well for my time.



That seems to work much better! So now we can parse the completion string and get the information out.

In [23]:
parse_completion(completion)

{'sentences': [('positive',
   'They were very professional and compensated me well for my time.')],
 'num_positive': 1,
 'num_negative': 0,
 'num_neutral': 0,
 'num_sentences': 1}

### Let's feed it some more reviews and see what the model does

In [24]:
prediction_list = []
for predict_item in to_predict_list:
    ## Generate prompt with examples
    few_shot_prompt = few_shot_examples + predict_item['prompt'] ## Notice how we include the same examples for every review. 

    ## Generate prediction
    result = openai.Completion.create(
        model = 'davinci', 
        prompt = few_shot_prompt,
        stop = ["<|endoftext|>"],
        max_tokens =  1000,
        temperature= 0.7
    )

    ## Parse completion
    completion = result['choices'][0]['text']
    completion_dict = parse_completion(completion)

    ## Store prediction
    prediction_list.append({
        'prompt' : predict_item['prompt'],
        'completion' : completion,
        'completion_dict' : completion_dict
    })

In [25]:
for item in prediction_list:
    print(item['prompt'].strip())
    print(item['completion'].strip())
    print('-------') 

I had a great time working with Chairlift Brothers! They were very professional and compensated me well for my time. I would definitely recommend them to anyone looking for a great chairlift company.
####
<positive> They were very professional and compensated me well for my time.
-------
I would not recommend working for Chairlift Brothers. They do not pay well and the hours are long. The work is also very physical and challenging, so it's not for everyone. There are better companies out there that will pay you more for your time and effort.
####
<negative> They do not pay well and the hours are long.
<negative> The work is also very physical and challenging, so it's not for everyone.
-------
The Chairlift Brothers company is terrible. They don't pay their employees well, and the work is extremely demanding. The company is also very disorganized, and it's hard to get anything done. Overall, I would not recommend working for this company.
####
<negative> The work is extremely demanding.

**Conclusion:** When running all the examples through the model we can see that the model consistently generates a completion that follows our structure, however, it often extracts sentences that do not relate to compensation. This, again, is not particularly surprising given that we only gave the model three examples.  

We could try to solve this by changing our prompt+comletion design. For example, we could try a prompt design where the completion includes every sentence, with an indicator as to whether the sentence relate to compensation or not (see the example below). As mentioned in the blog post, it is more of an art than a science and you have to try different design to find one that works best for your problem! 

Alternative design:

```
Plumbing Co is a great company to work for! The compensation is great and above the industry standard. The benefits are also very good. The company is very fair and treats its employees well. I would definitely recommend Plumbing Co to anyone looking for a great place to work.
####
<0><>---Plumbing Co is a great company to work for!
<1><positive>---The compensation is great and above the industry standard.
<1><positive>---The benefits are also very good.
<0><>---The company is very fair and treats its employees well.
<0><>---I would definitely recommend Plumbing Co to anyone looking for a great place to work.
```

The other solution would be to give the model through fine-tuning, which is shown next.

# Fine-tuning example
-------------------

Fine-tuning a GPT model is the most powerful way to improve the performance of the model, but it also requires more work. 

Below will just demonstrate a very basic fine-tuning setup, for the full details and discussion see the documentation: https://beta.openai.com/docs/guides/fine-tuning

### Step 1: Generate training data file

OpenAI requires the training data to be in a specific format, but they provide helper functions to generate this format. This what I am doing below.

In [42]:
training_csv = pd.DataFrame(training_list)
training_csv.to_csv(r'F:\training_data.csv', index=False)

To generate the training data we run the following command in the terminal:

```bash
openai tools fine_tunes.prepare_data -f F:\training_data.csv
```

This will generate a `.jsonl` file in the same directory as the `.csv` file which we can then use to train our model.

### Step 2: Fine tune the model

To start fine-tuning we run the following command in the terminal:

```bash
openai api fine_tunes.create -t "F:\training_data_prepared.jsonl" -m curie
```

**A few important notes:**

**Note #1:** OpenAI provides various models of different sizes. Davinci is the largest model available and is expensive to fine-tune. For most tasks you are better off using one of the smaller models, such as Curie or Babbage. These models are easier to fine-tune, cheaper, and have a higher inference throughput.  

**Note #2:** OpenAI recommends at least a few hundred examples for fine-tuning. Here I only give the model 10 examples for the sake of demonstration. 

**Note #3:** There is a cost associated with fine-tuning a model through OpenAI. You will be provided with a cost estimate when running the `openai api fine_tunes.create` command. For a Curie or Babbage model the fine-tuning costs are generally managable (i.e., between $5 and $50 for most tasks). 

---- 
There is a fine-tuning queue, so you might have to wait a while for your fine-tuning to start. Once you fine-tune is complete you will be provided with a model id, which you can use to interact with the model through the API and the Playground. 

In this case my fine-tuned model is called `curie:ft-personal-2022-04-14-03-40-04`   
(Sidenote, OpenAI recently added the option to give your model a more descriptive name, which I definitely recommend). 


### Step 3: Generate predictions using our custom model

The first time you submit a completion request to a custom model it might take a minute for the model to load and initailize.     
So you might see your first request time-out, don't worry, just try it again after about a minute and everything should work.

In [43]:
model_id = 'curie:ft-personal-2022-04-14-03-40-04'

#### Run a quick test prompt

In [12]:
test_prompt = 'The weather was great. The benefits were ok and nothing special. I did like the sandwiches, especially the cheese.' + '\n####\n' 
## Note, it is important that we add '\n####\n' to the end of the prompt, because that is what we trained the model to look for!

In [48]:
result = openai.Completion.create(
    model = model_id,  ## <--- We change this with our custom model ID
    prompt = test_prompt,
    stop = ["<|endoftext|>"],
    max_tokens =  500,
    temperature= 0.7
)

In [50]:
completion = result['choices'][0]['text']
print(completion.strip())

<negative> The benefits were ok and nothing special.


#### Generate a prediction for our reviews using our custom model

In [51]:
prediction_list = []
for predict_item in to_predict_list:
    ## Generate prompt without the examples
    ft_prompt = predict_item['prompt']

    ## Generate prediction
    result = openai.Completion.create(
        model = model_id,  ## <--- We change this with our custom model ID
        prompt = ft_prompt,
        stop = ["<|endoftext|>"],
        max_tokens =  500,
        temperature= 0.7
    )

    ## Parse completion
    completion = result['choices'][0]['text']
    completion_dict = parse_completion(completion)

    ## Store prediction
    prediction_list.append({
        'prompt' : predict_item['prompt'],
        'completion' : completion,
        'completion_dict' : completion_dict
    })

In [52]:
for item in prediction_list:
    print(item['prompt'].strip())
    print(item['completion'].strip())
    print('-------')

I had a great time working with Chairlift Brothers! They were very professional and compensated me well for my time. I would definitely recommend them to anyone looking for a great chairlift company.
####
<positive> They were very professional and compensated me well for my time.
-------
I would not recommend working for Chairlift Brothers. They do not pay well and the hours are long. The work is also very physical and challenging, so it's not for everyone. There are better companies out there that will pay you more for your time and effort.
####
<negative> The work is also very physical and challenging, so it's not for everyone.
-------
The Chairlift Brothers company is terrible. They don't pay their employees well, and the work is extremely demanding. The company is also very disorganized, and it's hard to get anything done. Overall, I would not recommend working for this company.
####
<neutral> The company is also very disorganized, and it's hard to get anything done.
-------
I don'

**Conclusion:** The predictions are similar to the few-shot example, however, we switched from the Davinci model to the Curie model (which is much smaller), so the fact that this still works with only 10 examples is quite impressive. Creating prediction using our fine-tuned Curie model is also much faster and cheaper relative to using few shot with Davinci. If you'd fine-tune a Curie model with a few hundred examples I'd expect the model to yield usable and reliable results.

# Good luck with your own GPT models and fine-tuning!